In [87]:
import numpy as np
import math

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.threebody import flybys
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D, OrbitPlotter3D
from poliastro.util import norm
import poliastro.twobody.propagation as Propagation

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [88]:
launch_date = Time("2027-06-20",scale='tdb')
DSM = Time("2028-07-18",scale='tdb')
earth_flyby = Time("2029-08-28",scale='tdb')
saturn_arrival = Time("2035-05-25",scale='tdb')

In [89]:
#create orbits
o_e1 = Orbit.from_body_ephem(Earth,launch_date)
o_e2 = Orbit.from_body_ephem(Earth,earth_flyby)
o_s  = Orbit.from_body_ephem(Saturn,saturn_arrival)

In [99]:
#perform c3
C_3 = 29.9*u.km**2/u.s**2

dv_launchC3 = (C_3 ** 0.5)* o_e1.v / norm(o_e1.v)

In [100]:
dv_launchC3

<Quantity [ 5.4642812 ,-0.18711429,-0.08135823] km / s>

In [101]:
c3Burn = Maneuver.impulse(dv_launchC3)

In [102]:
e_dsm = o_e1.apply_maneuver(c3Burn)

In [103]:
e_dsm.plot()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [135]:
e_dsm.p.to(u.AU)

<Quantity 1.4187793422280313 AU>

In [104]:
#propagate to dsm
o_dsm_minus = e_dsm.propagate(e_dsm.period / 2)

In [105]:
#determine the deltav needed to hit earth again
dms_e_time = earth_flyby - o_dsm_minus.epoch

In [106]:
dms_e_time.to(u.day)

<Quantity 404.37670876974187 d>

In [107]:
(v_dsm_plus, v_eflyby_minus), = izzo.lambert(Sun.k, o_dsm_minus.r, o_e2.r, dms_e_time)

In [108]:
v_dsm_plus

<Quantity [-13.99366241,  0.50731336,  0.21988115] km / s>

In [109]:
v_eflyby_minus

<Quantity [ 24.8647495 , 21.91794907,  9.49923785] km / s>

In [110]:
#determine delta v of dsm
dsm_dv = v_dsm_plus - o_dsm_minus.v

In [111]:
norm(dsm_dv)

<Quantity 1.2354443241575022 km / s>

In [112]:
#apply manourver
dsm_man = Maneuver.impulse(dsm_dv)
o_dsm_plus = o_dsm_minus.apply_maneuver(dsm_man)

In [113]:
#propagate
dsm_e_time = earth_flyby-DSM;
o_eflyby_minus = o_dsm_plus.propagate(dsm_e_time)

In [114]:
#compute requirement for earth to saturn
e_s_time = saturn_arrival-earth_flyby;
(v_eflyby_plus, v_saturn_minus),=izzo.lambert(Sun.k, o_e2.r, o_s.r, e_s_time)
o_es = Orbit.from_vectors(Sun, o_e2.r,v_eflyby_plus)


In [115]:
v_eflyby_plus

<Quantity [ 24.78563671, 28.1762668 , 13.04183929] km / s>

In [116]:
earth_flyby_dv = v_eflyby_plus-v_eflyby_minus

In [121]:
norm(earth_flyby_dv)

<Quantity 7.191858207911887 km / s>

In [127]:
print(norm(dsm_dv))
print(norm(earth_flyby_dv))
print(norm(v_saturn_minus-o_s.v))

1.2354443241575022 km / s
7.191858207911887 km / s
5.847156150528917 km / s


In [126]:
norm(v_eflyby_plus-o_e2.v)

<Quantity 13.207327885587445 km / s>

In [119]:
op=OrbitPlotter2D()
op.plot(o_e1)
op.plot(o_e2)
op.plot(o_s)
op.plot(e_dsm)
op.plot(o_dsm_plus)
op.plot(o_es)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [120]:
op=OrbitPlotter3D()
op.plot(o_e1)
op.plot(o_e2)
op.plot(o_s)
op.plot(e_dsm)
op.plot(o_dsm_plus)
op.plot(o_es)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/propagation.py:232: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'line': {'color': 'rgb(31, 119, 180)', 'dash': 'dash', 'width': 5},
             …

In [131]:
o_e1.inc-o_s.inc

<Quantity 0.01534915305115464 rad>

In [130]:
o_s.inc

<Quantity 0.3936174906576033 rad>